In [3]:
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from random import sample
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
####  seleccionar items distribucion 

In [4]:
user_interact = pd.read_csv('file:///home/dot/Documents/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
                  sep='\t',
                 names = ['UserId','TimeStamp','ArtId','ArtName','TraId','TraName'])

In [5]:
count = user_interact.groupby('ArtId').agg({'UserId':'count'}).reset_index()

In [10]:
count.UserId.describe()

count    107295.000000
mean        172.403271
std        1415.661208
min           1.000000
25%           2.000000
50%           7.000000
75%          35.000000
max      115099.000000
Name: UserId, dtype: float64

In [8]:
plt.bar(count.index,count.ArtId)

<BarContainer object of 107295 artists>

Error in callback <function flush_figures at 0x7f11fd2bb170> (for post_execute):


KeyboardInterrupt: 

In [ ]:
user_interact = user_interact[pd.to_datetime(user_interact.TimeStamp) > pd.to_datetime('2005-01-01 00:000').tz_localize('UTC')]

In [ ]:
UserId = sample(list(user_interact.UserId.unique()),int(len(list(user_interact.UserId.unique()))*(0.8)))

In [ ]:
data_sample = user_interact[user_interact.UserId.isin(UserId)]

## Artista
* clean dataset

In [ ]:
class recommender_model():
    '''
    df = query txt users
    rating_matrix =  
    '''
    def __init__(self,df):
        self.data_sample = df
        
    def rating_matrix(self):
        art = ((self.data_sample[['ArtName','ArtId']].dropna()).drop_duplicates()).drop_duplicates(['ArtId'])
        table = self.data_sample[['UserId','ArtId']]
        ratings_artistas = table.groupby(['UserId', 'ArtId']).size().reset_index(name = 'frecuencia')
        ratings_artistas['log_frecuencia'] = np.log10(ratings_artistas['frecuencia'])
        self.ratings_artistas = ratings_artistas
        
    def K_neigbors(self):
        reader = Reader(rating_scale = (min(self.ratings_artistas['log_frecuencia']),
                                        max(self.ratings_artistas['log_frecuencia']) ))
        surprise_dataset = Dataset.load_from_df(self.ratings_artistas[['UserId','ArtId','log_frecuencia']],
                                                reader)
        train_set, test_set = train_test_split(surprise_dataset, test_size=.2)
        k,value = [],[]
        for i in range(5,100,5):
            sim_options = {'name': 'cosine',
                           'user_based': False}

            algo = KNNBasic(k = i, min_k = 2, sim_options = sim_options)
            algo.fit(trainset = train_set)
            test_predictions = algo.test(test_set)
            value.append(accuracy.rmse(test_predictions, verbose = True))
            k.append(i)
        n = [i for i in range(len(value)) if value[i] == min(value)]
        self.k = k[n[0]]
        self.surprise_dataset = surprise_dataset
    def model (self):
        rating_data = self.surprise_dataset.build_full_trainset()
        test = rating_data.build_anti_testset()
        sim_options = {'name': 'cosine',
                       'user_based': True} 
        algo = KNNBasic(k=self.k, min_k=2, sim_options=sim_options)
        algo.fit(rating_data)
        filename = 'recomender_model_colaborative.sav'
        joblib.dump(algo, filename)
        

In [ ]:
model = recommender_model(user_interact)

In [ ]:
model.rating_matrix()

In [ ]:
model.K_neigbors()

In [ ]:
art = ((data_sample[['ArtName','ArtId']].dropna()).drop_duplicates()).drop_duplicates(['ArtId'])
#art.to_csv('art_table.csv',sep=',',index=False)
table = data_sample[['UserId','ArtId']]
ratings_artistas = table.groupby(['UserId', 'ArtId']).size().reset_index(name = 'frecuencia')
ratings_artistas['log_frecuencia'] = np.log10(ratings_artistas['frecuencia'])

In [ ]:
reader = Reader( rating_scale = ( min(ratings_artistas['log_frecuencia']), max(ratings_artistas['log_frecuencia']) ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings_artistas[[ 'UserId', 'ArtId', 'log_frecuencia' ]], reader )


In [ ]:
train_set, test_set = train_test_split(surprise_dataset, test_size=.2)

In [ ]:
k = []
value = []
for i in range(1,100,5):
    sim_options = {'name': 'cosine',
                   'user_based': True}

    algo = KNNBasic(k = i, min_k = 2, sim_options = sim_options)
    algo.fit(trainset = train_set)
    test_predictions = algo.test(test_set)
    value.append(accuracy.rmse(test_predictions, verbose = True))
    k.append(i)
n = [i for i in range(len(value)) if value[i] ==min(value)]

In [ ]:
rating_data=surprise_dataset.build_full_trainset()
test = rating_data.build_anti_testset()
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=k[n[0]], min_k=2, sim_options=sim_options)
algo.fit(rating_data)


In [ ]:
import joblib
filename = 'finalized_model.sav'
joblib.dump(algo, filename)

In [ ]:
test_p  =[test[i] for i in range(len(test)) if test[i][0] =='user_000002']

In [ ]:
user_prediction=algo.test(test_p)

In [ ]:
labels = ['movie id', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_prediction)), columns=labels)
df_predictions.sort_values(by='estimation', ascending = False).head(10)

In [ ]:
accuracy.rmse(test_predictions, verbose = True)

In [ ]:
loaded_model = joblib.load(filename)

In [ ]:
loaded_model.test(test_p)